# Tutorial

This notebook serves as a tutorial for how to use the Single Particle Model Code. First, begin by running the next cell in order to import all necessary packages for executing the code within this jupyter notebook.

In [84]:
# Import necessary packages
%matplotlib tk
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label
from netCDF4 import Dataset
import netCDF4 as NC
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (StrMethodFormatter, AutoMinorLocator)
from matplotlib.animation import FuncAnimation

$\bf{\mbox{Generating the input file}}$

This notebook allows the user to set the input parameters required for the half-SPM simulation.
A $\bf{\mbox{new simulation}}$ can be started by setting 'Starting from checkpoint file' to 'No' and setting the input parameters in the following cells. A NetCDF file called 'SPM_input.nc' will automatically be created which is then read by the programme.

The simulation can also be $\bf{\mbox{continued from a checkpoint}}$ file that is created during the first run by setting 'Starting from checkpoint file' to 'yes'. In that case, $\bf{\mbox{only the next two cells have to be run}}$ (provided the original 'input.nc' file still exists) and no further parameters have to be set.

In [85]:
Checkpoint_ =  HBox([Label('Starting from checkpoint file'), widgets.Select(options=['Yes', 'No'], value='No', disabled=False)])
Sim_steps_c = HBox([Label('No. of simulation steps'), widgets.BoundedIntText(value=1000, min=1, max=10**9, disabled=False)])
Out_steps_c = HBox([Label('Output written every [n] steps'), widgets.BoundedIntText(value=5, min=1, max=10**9, disabled=False)])
display(Checkpoint_, Sim_steps_c, Out_steps_c)

In [86]:
# Checkpoint check
if Checkpoint_.children[1].value == 'Yes':
        Checkpoint = 1
        rootgrp = NC('SPM_input.nc', 'r+', format='NETCDF4')
        rootgrp['checkpoint'][:] = Checkpoint
        rootgrp['sim_steps'][:] = Sim_steps_c.children[1].value 
        rootgrp['out_steps'][:] = Out_steps_c.children[1].value 
        rootgrp.close()
else:
        Checkpoint = 0

The following cell allows the user to set the input parameters within a suggested range, provided that a new simulation run is set up. The current default values are taken from Chen2020 (https:/doi.org/10.1149/1945-7111/ab9050) and are for a NCM (Nickel-Cobalt-Manganese) positive electrode.

The unresrticted option to set the input parameters exists in the following cell, however this is recommended for experienced users and only physical valid values should be set.

The remaining cells extract the set input parameters, convert them to SI units and creates a NetCDF input file that can be read by the programme.

In [109]:
# set input parameter within given ranges
Temp_ = HBox([Label('Temperature [°C]'), widgets.FloatSlider(min=-20.0, max=50.0, value=21.0, step=1)])
Rad_ = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatSlider(min=3.22, max=7.22, value=5.22, step=0.01)])
Thick_ = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatSlider(min=50.0, max=100.0, value=75.6, step=0.1)])
Rr_coef_ = HBox([Label('Reaction rate coefficent[$Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatSlider(min=1.5, max=6.5, value=3.42, step=0.01)])
Dif_coef_ = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatSlider(min=0, max=100, value=1.48, step=0.01)])
Max_c_ = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatSlider(min=0.0, max=100000.0, value=51765.0, step=1)])
SOC_ = HBox([Label('State of charge [%]'), widgets.FloatSlider(min=0, max=100, value=0, step=1)])
Current_ = HBox([Label('Applied current [A]'), widgets.FloatSlider(min=-20.0, max=20.0, value=5.0, step=0.1)])
Vol_per_ = HBox([Label('Active material volume fraction [%]'), widgets.FloatSlider(min = 0.0, max=100.0, value=66.5, step=0.1)])
Area_ = HBox([Label('Electrode plate area [$m^2$]'), widgets.FloatSlider(min=0.001, max=1.0, value=0.1027, step=0.001)])
Sim_steps_ = HBox([Label('No. of simulation steps'), widgets.BoundedIntText(value=1000, min=1, max=10**9, disabled=False)])
Dt_ = HBox([Label('Time step (s)'), widgets.FloatSlider(min=0.01, max=100.0, value=2.0, step=0.01)]) 
Out_steps_ = HBox([Label('Output written every [n] steps'), widgets.BoundedIntText(value=5, min=1, max=10**9, disabled=False)])
Space_steps_ = HBox([Label('No. of space steps'), widgets.FloatSlider(min=2, max=1000, value=20, step=1)])
Volt_do_ =  HBox([Label('Output voltage data'), widgets.Select(options=['Yes', 'No'], value='Yes', disabled=False)])

display(Temp_, Rad_, Thick_, Rr_coef_, Dif_coef_, Max_c_, SOC_, Current_, Vol_per_, Area_, Sim_steps_, Dt_, Out_steps_, Space_steps_, Volt_do_)

In [110]:
# unbounded boxes to set parameters outside of suggested ranges if wanted
Temp_ = HBox([Label('Temperature [°C]'), widgets.FloatText(value=Temp_.children[1].value, disabled=False)])
Rad_ = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatText(value=Rad_.children[1].value, disabled=False)])
Thick_ = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatText(value=Thick_.children[1].value, disabled=False)])
Rr_coef_ = HBox([Label('Reaction rate coeffic[ent($Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatText(value=Rr_coef_.children[1].value, disabled=False)])
Dif_coef_ = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatText(value=Dif_coef_.children[1].value, disabled=False)])
Max_c_ = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatText(value=Max_c_.children[1].value, disabled=False)])
Current_ = HBox([Label('Applied current [A]'), widgets.FloatText(value=Current_.children[1].value, disabled=False)])
Area_ = HBox([Label('Electrode area [$m^2$]'), widgets.FloatText(value=Area_.children[1].value, disabled=False)])
Dt_ = HBox([Label('Time step (s)'), widgets.FloatText(value=Dt_.children[1].value, disabled=False)])
Space_steps_ = HBox([Label('No. of space steps'), widgets.FloatText(value=Space_steps_.children[1].value, disabled=False)])

display(Temp_, Rad_, Thick_, Rr_coef_, Dif_coef_, Max_c_, Current_, Area_, Dt_, Space_steps_)

In [121]:
# saving the values set by the user and adjusting relevant parameters to be in SI units
Temp = Temp_.children[1].value + 273.15   #K
Rad = Rad_.children[1].value * 10**(-6)   #m
Thick = Thick_.children[1].value * 10**(-6)   #m
Rr_coef =  Rr_coef_.children[1].value
Dif_coef = Dif_coef_.children[1].value * 10**(-15)
Max_c = Max_c_.children[1].value
# Init_c derived from values in Chen2020:
# stoichiometry at 0% SOC = 0.2661, and at 100% SOC = 0.9084
# assuming linear behaviour we get x = mcx +c with c = 0.9084
# and m = 0.2661-0.9084 = -0.6423
Init_c = Max_c * ((SOC_.children[1].value/100)*(-0.6423) + 0.9084)
Iapp = Current_.children[1].value / Area_.children[1].value  # A/m^2
Vol_per = Vol_per_.children[1].value
Sim_steps = Sim_steps_.children[1].value
Dt = Dt_.children[1].value
Out_steps = Out_steps_.children[1].value
Space_steps = Space_steps_.children[1].value
if Volt_do_.children[1].value == 'Yes':
    Volt_do = 1
else:
    Volt_do = 0

In [122]:
#create new NetCDF file in 'writing mode' and 'NETCDF4 format'
rootgrp = Dataset('SPM_input.nc', 'w', format='NETCDF4')
#creating dimensions for vector holding all the input variables
temp_dim = rootgrp.createDimension('temp_dim', 1)
rad_dim = rootgrp.createDimension('rad_dim', 1)
thick_dim = rootgrp.createDimension('thick_dim', 1)
rr_coef_dim = rootgrp.createDimension('rr_coef_dim', 1)
dif_coef_dim = rootgrp.createDimension('dif_coef_dim', 1)
max_c_dim = rootgrp.createDimension('max_c_dim', 1)
init_c_dim = rootgrp.createDimension('init_c_dim', 1)
iapp_dim = rootgrp.createDimension('iapp_dim', 1)
vol_per_dim = rootgrp.createDimension('vol_per_dim', 1)
sim_steps_dim = rootgrp.createDimension('sim_steps_dim', 1)
dt_dim = rootgrp.createDimension('dt_dim', 1)
out_steps_dim = rootgrp.createDimension('out_steps_dim', 1)
space_steps_dim = rootgrp.createDimension('space_steps_dim', 1)
volt_do_dim = rootgrp.createDimension('volt_do_dim', 1)
checkpoint_dim = rootgrp.createDimension('checkpoint_dim', 1)
#creating variable 
temp = rootgrp.createVariable('temp', 'f8', ('temp_dim',))
rad = rootgrp.createVariable('rad', 'f8', ('rad_dim',))
thick = rootgrp.createVariable('thick', 'f8', ('thick_dim',))
rr_coef = rootgrp.createVariable('rr_coef', 'f8', ('rr_coef_dim',))
dif_coef = rootgrp.createVariable('dif_coef', 'f8', ('dif_coef_dim',))
max_c = rootgrp.createVariable('max_c', 'f8', ('max_c_dim',))
init_c = rootgrp.createVariable('init_c', 'f8', ('init_c_dim',))
iapp = rootgrp.createVariable('iapp', 'f8', ('iapp_dim',))
vol_per = rootgrp.createVariable('vol_per', 'f8', ('vol_per_dim',))
sim_steps = rootgrp.createVariable('sim_steps', 'i4', ('sim_steps_dim',))
dt = rootgrp.createVariable('dt', 'f8', ('dt_dim',))
out_steps = rootgrp.createVariable('out_steps', 'i4', ('out_steps_dim',))
space_steps = rootgrp.createVariable('space_steps', 'i4', ('space_steps_dim',))
volt_do = rootgrp.createVariable('volt_do', 'i4', ('volt_do_dim',))
checkpoint = rootgrp.createVariable('checkpoint', 'i4', ('checkpoint_dim',))
# attributes
rootgrp.description = 'Input parameters for SMP model'
temp.description = 'Temperature'
temp.units = 'K'
rad.description = 'Mean particle radius'
rad.units = 'm'
thick.description = 'Electrode thickness'
thick.units = 'm'
rr_coef.description = 'Reaction rate coefficient'
rr_coef.units = '$Am^{-2}(m^3mol^{-1})^{1.5}$'
dif_coef.description ='Diffusion coefficient'
dif_coef.units = '$m^2 s^{-1}$'
max_c.description = 'Maximum lithium concentration'
max_c.units = '$mol m^{-3}$'
init_c.description = 'Initial lithium concentration'
init_c.units = '$mol m^{-3}$'
iapp.description = 'Applied current density'
iapp.units = '$A/m^2$'
vol_per.description = 'Active material volume fraction'
vol_per.units = '%'
sim_steps.description = 'Total number of simultion steps'
sim_steps.units = 'unitless'
dt.description = 'Time step'
dt.units = 's'
out_steps.description = 'Output written every [n] number of steps'
out_steps.units = 'untiless'
space_steps.description = 'No. of space steps'
space_steps.units = 'unitless'
volt_do.description = 'Write voltage data'
volt_do.units = 'unitless'
checkpoint.description = 'Starting from checkpont file'
checkpoint.units = 'unitless'
# writing data to input_parameters variable
temp[0] = Temp
rad[0] = Rad
thick[0] = Thick
rr_coef[0] = Rr_coef
dif_coef[0] = Dif_coef
max_c[0] = Max_c
init_c[0] = Init_c
iapp[0] = Iapp
vol_per[0] = Vol_per
sim_steps[0] = Sim_steps
dt[0] = Dt
out_steps[0] = Out_steps
space_steps[0] = Space_steps
volt_do[0] = Volt_do
checkpoint[0] = Checkpoint
#closing the NetCDF file
rootgrp.close()

Option to perform uncertainty propagation on given input variables. A variable is ignored by setting its value equal to 0. To run this a input 'SMP_input.nc' file needs to be already present and only the following 3 cells have to be run.

In [87]:
# option to conduct uncertainty propagation
UQ_ = HBox([Label('Conduct uncertainty propagation'), widgets.Select(options=['Yes', 'No'], value='No', disabled=False)])
No_samples_ = HBox([Label('No. of samples [unitless]'), widgets.IntText(value=10, disabled=False)])
Temp_std_ = HBox([Label('Standard devidation (std) of temperature [°C]'), widgets.FloatText(value=21.0*0.05, disabled=False)])
Rad_std_ = HBox([Label('Std of mean particle radius [$\mu m$]'), widgets.FloatText(value=5.22*0.05, disabled=False)])
Thick_std_ = HBox([Label('Std of electrode thickness [$\mu m$]'), widgets.FloatText(value=75.6*0.05, disabled=False)])
Rr_coef_std_ = HBox([Label('Std of reaction reate coefficient [$Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatText(value=6.5*0.05, disabled=False)])
Dif_coef_std_ = HBox([Label('Std of diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatText(value=1.48*0.05, disabled=False)])
Init_c_std_ = HBox([Label('Std of initial concentration [$mol m^{-3}$]'), widgets.FloatText(value=47023.326*0.05, disabled=False)])
Vol_per_std_ = HBox([Label('Std of active material volume fraction [%]'), widgets.FloatText(value=66.5*0.05, disabled=False)])
Iapp_std_ = HBox([Label('Std of applied current [A]'), widgets.FloatText(value=5.0*0.05, disabled=False)])
display(UQ_, No_samples_, Temp_std_, Rad_std_, Thick_std_, Rr_coef_std_, Dif_coef_std_, Init_c_std_, Vol_per_std_, Iapp_std_)

In [123]:
# saving the set values
No_samples = No_samples_.children[1].value
Temp_std = Temp_std_.children[1].value + 273.15*0.05 #K
Rad_std = Rad_std_.children[1].value * 10**(-6)   #m
Thick_std = Thick_std_.children[1].value * 10**(-6)   #m
Rr_coef_std = Rr_coef_std_.children[1].value
Dif_coef_std = Dif_coef_std_.children[1].value * 10**(-15)
Init_c_std = Init_c_std_.children[1].value
Vol_per_std = Vol_per_std_.children[1].value
Iapp_std = Iapp_std_.children[1].value

In [124]:
if UQ_.children[1].value == 'Yes':
        rootgrp = Dataset('SPM_input.nc', 'r+', format='NETCDF4')
        # no_samples
        no_samples_dim = rootgrp.createDimension('no_samples_dim', 1)
        no_samples = rootgrp.createVariable('no_samples', 'i4', ('no_samples_dim',))
        no_samples.description = 'No. of samples'
        no_samples.units = 'unitless'
        no_samples[0] = No_samples
        # Temp_std
        if Temp_std != 0:
            temp_std_dim = rootgrp.createDimension('temp_std_dim', 1)
            temp_std = rootgrp.createVariable('temp_std', 'f8', ('temp_std_dim',))
            temp_std.description = 'Std of temperature'
            temp_std.units = 'K'
            temp_std[0] = Temp_std
        # Rad_std
        if Rad_std != 0:
            rad_std_dim = rootgrp.createDimension('rad_std_dim', 1)
            rad_std = rootgrp.createVariable('rad_std', 'f8', ('rad_std_dim',))
            rad_std.description = 'Std of mean particle radius'
            rad_std.units = 'm'
            rad_std[0] = Rad_std
        # Thick_std
        if Thick_std != 0:
            thick_std_dim = rootgrp.createDimension('thick_std_dim', 1)
            thick_std = rootgrp.createVariable('thick_std', 'f8', ('thick_std_dim',))
            thick_std.description = 'Std of electrode thickness'
            thick_std.units = 'm'
            thick_std[0] = Thick_std
        # Rr_std
        if Rr_coef_std != 0:
            rr_coef_std_dim = rootgrp.createDimension('rr_coef_std_dim', 1)
            rr_coef_std = rootgrp.createVariable('rr_coef_std', 'f8', ('rr_coef_std_dim',))
            rr_coef_std.description = 'Std of reaction rate coefficient'
            rr_coef_std.units = '$Am^{-2}(m^3mol^{-1})^{1.5}$'
            rr_coef_std[0] = Rr_coef_std
        # Dif_coef
        if Dif_coef_std != 0:
            dif_coef_std_dim = rootgrp.createDimension('dif_coef_std_dim', 1)
            dif_coef_std = rootgrp.createVariable('dif_coef_std', 'f8', ('dif_coef_std_dim',))
            dif_coef_std.description = 'Std of diffusion coefficient'
            dif_coef_std.units = '$m^2 s^{-1}$'
            dif_coef_std[0] = Dif_coef_std
        # Init_c_std
        if Init_c_std != 0:
            init_c_std_dim = rootgrp.createDimension('init_c_std_dim', 1)
            init_c_std = rootgrp.createVariable('init_c_std', 'f8', ('init_c_std_dim',))
            init_c_std.description = 'Std of initial concentration'
            init_c_std.units = '$mol m^{-3}$'
            init_c_std[0] = Init_c_std
        # Vol_per_std
        if Vol_per_std != 0:
            vol_per_std_dim = rootgrp.createDimension('vol_per_std_dim', 1)
            vol_per_std = rootgrp.createVariable('vol_per_std', 'f8', ('vol_per_std_dim',))
            vol_per_std.description = 'Std of active material volume fraction'
            vol_per_std.units = '%'
            vol_per_std[0] = Vol_per_std
        # Iapp_std
        if Iapp_std != 0:
            iapp_std_dim = rootgrp.createDimension('iapp_std_dim', 1)
            iapp_std = rootgrp.createVariable('iapp_std', 'f8', ('iapp_std_dim',))
            iapp_std.description = 'Std of initial concentration'
            iapp_std.units = 'A'
            iapp_std[0] = Iapp_std
        rootgrp.close()

$\bf{\mbox{Compiling and running the main program}}$

The following cells are used to compile all relevant files and the run the main program. The first cell executes a terminal command within the directory this file is located. In this case, the cell runs the "make" command which compiles all relevent files. The following cell executes "make exe" which runs the executable created by the first cell. The laste cell of this section executes "make clean" which deletes previously generated files and allows for the first cell to be run again. This cell should only be run if there are any errors with the previous cells.

In [95]:
# Run this cell to compile the code and create the executables
!make

gfortran  `nf-config --fflags`  -c input_output_netcdf.f90 `nf-config --flibs` -llapack  -o input_output_netcdf.o
gfortran  `nf-config --fflags`  -c pde.f90 `nf-config --flibs` -llapack  -o pde.o
gfortran  `nf-config --fflags`  input_output_netcdf.o pde.o main.f90 `nf-config --flibs` -llapack  -o test.out
rm -f *.o *.mod
chmod +x uq_code/sens_ana.sh
chmod +x uq_code/up_code.sh
Crank-Nicolson solver ready
Input successful
Compilation sucessful


In [125]:
# Run this cell to execute the code and generate data
!make exe

Running Serial code


In [13]:
# If errors are being displayed, run this cell to delete all files created by make. 
# Next, run the first and second cells of this section again
!make clean

rm -f *.o *.mod *.nc *chp test.out
rm -f uq_code/*.nc uq_code/*.csv
rm -r uq_code/data_store_sens
rm: cannot remove 'uq_code/data_store_sens': No such file or directory
make: *** [Makefile:83: clean] Error 1


$\bf{\mbox{Plotting}}$

Run the cell below to display the animation of the data that was generated. The cell displays plots of Lithium concentration across the particle radius as a function of time. It also displays the concentration across the particle as a coloured contour plot as well as plot of the ouput voltage against time step.

In [66]:
# Creates a pop up window with the plotted data
plt.close()
# reading in the NetCDF data from SP_output.nc
dat = NC.Dataset("SP_output.nc", "r", format="NETCDF4")
do_volt = dat['volt_do'][:][0]
dt = dat['dt'][:][0]

#creating figure and axes dependent on whether voltage data is to be written or not
if do_volt == 1:
	fig = plt.figure(figsize=(10,8))
	ax1 = plt.subplot2grid((2,2),(0,0))
	ax2 = plt.subplot2grid((2,2),(0,1))
	ax3 = plt.subplot2grid((2,2),(1,0), colspan=2)
else:
	fig = plt.figure(figsize=(10,4))
	ax1 = plt.subplot2grid((1,2),(0,0))
	ax2 = plt.subplot2grid((1,2),(0,1))


# Subplot 1 - Animtion of lithium concentration 
# accessing the 2d concentration data and creating a time and space axis
c = dat['conc'][:]

t_steps = np.shape(c)[0]
r_steps = dat['space_steps'][:][0]
rad = dat['rad'][:][0]*10**(6)
time_axis = np.linspace(0, t_steps-1, t_steps-1).astype(int)
x_axis = np.linspace(0, rad, r_steps, endpoint=True)

# animation of the concentration data 
# the main parts of the  animation code were taken from:
# https://brushingupscience.com/2016/06/21/matplotlib-animations-the-easy-way/

intervaltime = 0.5   #time between each animations step in ms

# plotting the first graph which is a 2D line of lithium ion concentration
# with respect to the particle radius
line, = ax1.plot(x_axis, c[0])

# function evolving the animation, which is called each frame 
# the x-axis is the particle radius, so only the x-axis data
# i.e. the lithium concantration changes with simulation time
def animate(t):
    line.set_ydata(c[t,:])
    return line,

# creating the animation using matplotlib's FunAnimation
# the arguments to be passed:
# figure of the graph  - fig from fix,ax = plt.subplots())
# function evolving the animation - animate(t)
# interval time between each timesetep - intervaltime
# frames, the array to iterate over - time_axis
# blit - blitting set to true
animation = FuncAnimation(fig, animate, interval=intervaltime, frames=time_axis, blit=True)

# customise the graph with axes labels, major and minor ticks, labels etc.
ax1.set_xlabel('Distance from particle centre [$\mu m$]', size=8)
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(axis='x', labelsize=7)
ax1.set_ylabel('Lithium concentration $mol*m^{-3}$', size=8)
ax1.set_ylim(np.min(c)-1*10**(-9), np.max(c)+1*10**(-9))
ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(axis='y', labelsize=7)
ax1.ticklabel_format(axis='both', style="sci", useMathText=True)
ax1.xaxis.offsetText.set_fontsize(7)
ax1.yaxis.offsetText.set_fontsize(7)
ax1.set_title('Lithium Concentration Across Particle Radius', size=10, pad=15.0)

# Subplot 2 - pcolorplot of concentration data

# most of the required data is the same as in subplot 1
# get the discretisation steps of the radius
dr = rad/(r_steps-1)
 
# reversed virdis colormap, values over the range are black, under goes white
colour = cm.get_cmap('viridis_r').copy()
colour.set_under(color='w')
colour.set_over(color='k')
max_colour = np.max(c)
min_colour = np.min(c)
colour_range = max_colour - min_colour
ticklist = np.linspace(min_colour, max_colour, 6)

#create grid for plotting
#invert arrays to make sure smallest circle is on top
rs = np.linspace(0,rad,r_steps,endpoint=True)
rs = rs[::-1]
cinv = c[:,::-1]

#draw patches of circles
#zorder to ensure smallest circle is on top
patches = []
for i, r in enumerate(rs):
    circle = mpl.patches.Circle((0,0), r, zorder=i)
    patches.append(circle)

#set limits for colourmap and set colours of circles for initial plot
#add circles to collection    
p = mpl.collections.PatchCollection(patches, cmap=colour)
p.set_clim([min_colour,max_colour])
colours = np.array(cinv[0,:])
p.set_array(colours)
ax2.add_collection(p)

#set figure labels
ax2.set_xlim(-rad,rad)
ax2.set_ylim(-rad,rad)
ax2.set_xlabel('Distance from particle centre [$\mu m$]', size=8)
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax2.tick_params(axis='x', labelsize=7)
ax2.set_ylabel('Distance from particle centre [$\mu m$]', size=8)
ax2.yaxis.set_minor_locator(AutoMinorLocator())
ax2.tick_params(axis='y', labelsize=7)
ax2.ticklabel_format(axis='both', style="sci", useMathText=True)
ax2.xaxis.offsetText.set_fontsize(7)
ax2.yaxis.offsetText.set_fontsize(7)
ax2.set_title('Contour Plot of Concentration inside Particle', size=10, pad=15.0)
cbar = ax2.figure.colorbar(p, ax=ax2, cmap=colour, ticks=ticklist)
cbar.ax.tick_params(labelsize=7)
cbar.ax.set_ylabel('Lithium concentration $mol*m^{-3}$', size=8)
cbar.ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.10f}"))

# animation function for the circle plot animation
#only changes the colours of the circles without redrawing them
# the concentration data for each timestep is feeded into a colour map
# the inteval and frames are the same as in subplot 1 and blitting is set to true
def animate_pcol(t):
    colours = np.array(cinv[t,:])
    p.set_array(colours)
    
    return p,

animation_pcol = FuncAnimation(fig, animate_pcol, interval=intervaltime, frames=time_axis, blit=True)

# Subplot 3 - plot of voltage output if do_volt set to true
if do_volt == 1:

	# accessing the voltage data and setting the number of time steps as x-axis
	volt = dat['volt'][:][:,0]
	time = np.linspace(0, t_steps, t_steps)

	# plotting the 2D graph and customising it
	ax3.plot(time*dt, volt)
	ax3.set_xlabel('Time [s]', size=8)
	ax3.xaxis.set_minor_locator(AutoMinorLocator())
	ax3.tick_params(axis='x', labelsize=7)
	ax3.set_ylabel('Voltage [V]', size=8)
	ax3.yaxis.set_minor_locator(AutoMinorLocator())
	ax3.tick_params(axis='y', labelsize=7)
	ax3.ticklabel_format(axis='both', style="sci", useMathText=True)
	ax3.xaxis.offsetText.set_fontsize(7)
	ax3.yaxis.offsetText.set_fontsize(7)
	ax3.set_title('Voltage output', size=10, pad=15.0)

	# animation of an curser that moves with the time axis with the same interval and frames
	# as the other two animations
	vl = ax3.axvline(time[0]*dt, color='black', linestyle=':')
	ax3.set_xlim()

	def animate_t_bar(t):
		vl.set_xdata(time[t]*dt)
		return vl,

	animation_t_bar = FuncAnimation(fig, animate_t_bar, interval=intervaltime, frames=time_axis, blit=True)

plt.draw()
plt.tight_layout()
plt.show()

dat.close()

$\bf{\mbox{Sensitivity Analysis}}$

The following cells run and then display the sensitivity analysis. In order to display multiple plots simply run the plotting cells one after another without closing the previously pop up window.

In [149]:
# Perform sensitivity analysis, following cell visualises output
!make sensitive

(cd ./uq_code && ./sens_ana.sh False)
SENSITIVITY ANALYSIS

Preparing database store
Getting input Parameters and generating data
Generating input files
1/10
Running Serial code
2/10
Running Serial code
3/10
Running Serial code
4/10
Running Serial code
5/10
Running Serial code
6/10
Running Serial code
7/10
Running Serial code
8/10
Running Serial code
9/10
Running Serial code
10/10
Running Serial code
Visualising Results


In [152]:
# Creates pop up window that visualises the sensitivity analysis
#Set mean of variables here
params = ['temp','rad','thick','rr_coef','dif_coef','init_c','max_c','vol_per','iapp']

#Import the original parameters from the original input file and save to a vector
dat_inp = NC.Dataset("uq_code/data_store_sens/SPM_input_ori.nc", "r", format="NETCDF4")

mu = np.array([dat_inp['temp'][:][0],
               dat_inp['rad'][:][0],
               dat_inp['thick'][:][0],
               dat_inp['rr_coef'][:][0],
               dat_inp['dif_coef'][:][0],
               dat_inp['init_c'][:][0],
               dat_inp['max_c'][:][0],
               dat_inp['vol_per'][:][0],
               dat_inp['iapp'][:][0]])

sim_steps = dat_inp['sim_steps'][:][0]
dt = dat_inp['dt'][:][0]

#Auxilliary 2D array of variable means, identical columns
Mu_s = np.tile(mu.reshape((9,1)),(1,sim_steps))

#Set value of perturbation (eps) here
#Set h as a matrix for element-wise division, identical columns with elements = h_i
eps = 1e-6
h_i = (eps*mu).reshape((9,1))
h = np.tile(h_i,(1, sim_steps))

#Obtain mean voltage curve V_0 here, each row identical
#9 rows representing variables
#columns represent time steps
V_0 = np.zeros((9,sim_steps))
dat_mu = NC.Dataset("uq_code/data_store_sens/SP_output_0.nc", "r", format="NETCDF4")
volt_0 = np.array(dat_mu['volt'][:][:,0])
V_0 = np.tile(volt_0, (9, 1))

#Obtain perturbed voltages here
#9 rows representing variables
#Columns representing timesteps
Vs = np.zeros((9,sim_steps))

for i in range(1, 10, 1):
    dat = NC.Dataset(f"uq_code/data_store_sens/SP_output_{i}.nc", "r", format="NETCDF4")
    Vs[i-1, :] = np.array(dat['volt'][:][:,0])
    dat.close()

#Compute dV/dx as a function of time
dV_dx = np.divide((Vs - V_0),h)

#Compute absolute scaled sensitivity using element-wise multiplication
#each column i of the matrix represents the sensitivities for each paramter at time t=i
V_first_sensitivities = np.abs(Mu_s*dV_dx)

#Put the data into a data frame, and save to .csv
dat_fram = {'temp_sens': V_first_sensitivities[0, :],
            'rad_sens': V_first_sensitivities[1, :],
            'thick_sens': V_first_sensitivities[2, :],
            'rr_coef_sens': V_first_sensitivities[3, :],
            'dif_coef_sens': V_first_sensitivities[4, :],
            'init_c_sens': V_first_sensitivities[5, :],
            'max_c_sens': V_first_sensitivities[6, :],
            'vol_per_sens': V_first_sensitivities[7, :],
            'iapp_sens': V_first_sensitivities[8, :],
            'temp_dvdx': dV_dx[0, :],
            'rad_dvdx': dV_dx[1, :],
            'thick_dvdx': dV_dx[2, :],
            'rr_coef_dvdx': dV_dx[3, :],
            'dif_coef_dvdx': dV_dx[4, :],
            'init_c_dvdx': dV_dx[5, :],
            'max_c_dvdx': dV_dx[6, :],
            'vol_per_dvdx': dV_dx[7, :],
            'iapp_dvdx': dV_dx[8, :]}

cols = ['temp_sens', 'rad_sens', 'thick_sens', 'rr_coef_sens', 'dif_coef_sens', 'init_c_sens', 'max_c_sens', 'vol_per_sens', 'iapp_sens',
        'temp_dvdx', 'rad_dvdx', 'thick_dvdx', 'rr_coef_dvdx', 'dif_coef_dvdx', 'init_c_dvdx', 'max_c_dvdx', 'vol_per_dvdx', 'iapp_dvdx']
df = pd.DataFrame(dat_fram, columns=cols)
df.to_csv('sens_data.csv', index=False)


#plot as animation
fig, ax = plt.subplots(figsize=(10,6))

x = np.arange(0, 9, 1)

#time between frames
intervaltime = 0.5

#plot first frame
line, = plt.plot(x, V_first_sensitivities[:, 0], 'o-')

#definition of animation
def animate(i):
    line.set_ydata(V_first_sensitivities[:, i])
    time.set_text(('T=')+str(i*dt)+(' s'))
    return line, time,

#Design plot
ax.set_xticks(x)
ax.set_xticklabels(params, rotation=45)
ax.set_ylabel('Absolute scaled sensitivity of $V$')
ax.set_xlabel('Parameter')
ax.set_ylim(np.min(V_first_sensitivities)+10**(-9)-0.1, np.max(V_first_sensitivities)+1)
ax.grid()
ax.set_title('First Order Sensitivities Over Time')

time = ax.text(0.1,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5}, transform=ax.transAxes)

#plot animation
animate_sensitivities = FuncAnimation(fig, animate,interval=10, frames=range(1,sim_steps), blit=True)

dat_inp.close()
dat_mu.close()

plt.show()


$\bf{\mbox{Uncertainty Quantification}}$

The following cells run and then display the uncertainty quantification. In order to display multiple plots simply run the plotting cells one after another without closing the previously pop up window.

In [153]:
# Perform approximate uncertainty quantification using standard deviations and sensitivity analysis
!make uncer_from_sens

(cd ./uq_code && ./sens_ana.sh True)
SENSITIVITY ANALYSIS

Preparing database store
Getting input Parameters and generating data
Generating input files
1/10
Running Serial code
2/10
Running Serial code
3/10
Running Serial code
4/10
Running Serial code
5/10
Running Serial code
6/10
Running Serial code
7/10
Running Serial code
8/10
Running Serial code
9/10
Running Serial code
10/10
Running Serial code
Visualising Results
10


In [163]:
# Create pop up widow that visualises approximate uncertainty quatification
#Get the results from the mean parameters and save to an array
dat_mu = NC.Dataset(f"uq_code/data_store_sens/SP_output_0.nc", "r", format="NETCDF4")
volt_dat_mu = np.array(dat_mu['volt'][:][:,0])

#Get the number of time steps and the step length in order to create a time x axis in seconds
t_steps = dat_mu['sim_steps'][:][0]
dt = dat_mu['dt'][:][0]
x = (np.linspace(0, t_steps, t_steps).astype(int))*dt

#Plot the mean voltage
plt.plot(x, volt_dat_mu, color='black', label='Mean Voltage')
plt.title('Uncertainty in Voltage Calculation', pad=15.0)
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')
plt.grid()

std_V_dat = pd.read_csv('uq_code/data_store_sens/std_V_dat.csv')

std_V = np.array(std_V_dat['std_V'][:])
low_b = volt_dat_mu-(2*std_V)
up_b = volt_dat_mu+(2*std_V)
plt.fill_between(x, low_b, up_b, alpha=0.2, label='95% Confidence (SDs)', color='C3')
plt.plot(x, low_b, alpha=0.5, color='C3')
plt.plot(x, up_b, alpha=0.5, color='C3')

#Display the plot
plt.legend()

dat_mu.close()

plt.show()


In [157]:
!make uncertain

(cd ./uq_code && ./up_code.sh False)
UNCERTAINTY PROPAGATION

Getting input parameters and generating data
Preparing database store
Generating Input files
1/10
Running Serial code
2/10
Running Serial code
3/10
Running Serial code
4/10
Running Serial code
5/10
Running Serial code
6/10
Running Serial code
7/10
Running Serial code
8/10
Running Serial code
9/10
Running Serial code
10/10
Running Serial code
Visualising Results


In [164]:
# Creates pop up window that visualises the uncertainty quantification
#Read the input parameters from the original input file
inp_dat = NC.Dataset(f"uq_code/data_store_up/SPM_input_ori.nc", "r", format="NETCDF4")
num_samps = inp_dat['no_samples'][:][0]

#Get the results from the mean parameters and save to an array
dat_mu = NC.Dataset(f"uq_code/data_store_up/SP_output_0.nc", "r", format="NETCDF4")
volt_dat_mu = np.array(dat_mu['volt'][:][:,0])

#Get the number of time steps and the step length in order to create a time x axis in seconds
t_steps = dat_mu['sim_steps'][:][0]
dt = dat_mu['dt'][:][0]
x = (np.linspace(0, t_steps, t_steps).astype(int))*dt

#Create an array to store all the voltage data from each run
volt_array = np.empty((t_steps, (num_samps-1)))

#Open the files from the data base, extract the voltage and save to the array
for i in range(1, num_samps, 1):
    dat = NC.Dataset(f"uq_code/data_store_up/SP_output_{i}.nc", "r", format="NETCDF4")
    volt_array[:, (i-1)] = np.array(dat['volt'][:][:,0])
    dat.close()

#Calculate the 2.5th and 97.5th percentile of the voltage at each time step to get a 95% confidence
ana = np.percentile(volt_array, [2.5, 97.5], axis=1)

#Plot the range onto a graph
plt.fill_between(x, ana[0, :], ana[1, :], alpha=0.2, label='95% Confidence (random samps)', color='C0')
plt.plot(x, ana[0, :], color='C0', alpha=0.5)
plt.plot(x, ana[1, :], color='C0', alpha=0.5)

#Save the data to a data frame, and a .csv file for the user to be able to move to a different plotting software
dat_fram = {'mean volt': volt_dat_mu,
            '5th percentile': ana[0,:],
            '95th percentile': ana[1,:]}
df = pd.DataFrame(dat_fram, columns=['mean', '5th percentile', '95th percentile'])
df.to_csv('voltage_confidence_up.csv', index=False)

#Plot the mean voltage
plt.plot(x, volt_dat_mu, color='black', label='Mean Voltage')
plt.title('Uncertainty in Voltage Calculation', pad=15.0)
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')
plt.grid()

#If the user has performed the sensitivity analysis, and calculated uncertainty using dV_dx (see generate_inp_params.py)...
#...They can plot this data on top of the other uncertainty propagation data 
if (sys.argv[1] == 'True'):
    std_V_dat = pd.read_csv('./std_V_dat.csv')

    std_V = np.array(std_V_dat['std_V'][:])
    low_b = volt_dat_mu-(2*std_V)
    up_b = volt_dat_mu+(2*std_V)
    plt.fill_between(x, low_b, up_b, alpha=0.2, label='95% Confidence (SDs)', color='C3')
    plt.plot(x, low_b, alpha=0.5, color='C3')
    plt.plot(x, up_b, alpha=0.5, color='C3')

#Display the plot
plt.legend()

inp_dat.close()
dat_mu.close()

plt.show()



In [165]:
# Creates pop up window that visualises the uncertainty quantification and sensitivity analysis
#Read the input parameters from the original input file
inp_dat = NC.Dataset(f"uq_code/data_store_up/SPM_input_ori.nc", "r", format="NETCDF4")
num_samps = inp_dat['no_samples'][:][0]

#Get the results from the mean parameters and save to an array
dat_mu = NC.Dataset(f"uq_code/data_store_up/SP_output_0.nc", "r", format="NETCDF4")
volt_dat_mu = np.array(dat_mu['volt'][:][:,0])

#Get the number of time steps and the step length in order to create a time x axis in seconds
t_steps = dat_mu['sim_steps'][:][0]
dt = dat_mu['dt'][:][0]
x = (np.linspace(0, t_steps, t_steps).astype(int))*dt

#Create an array to store all the voltage data from each run
volt_array = np.empty((t_steps, (num_samps-1)))

#Open the files from the data base, extract the voltage and save to the array
for i in range(1, num_samps, 1):
    dat = NC.Dataset(f"uq_code/data_store_up/SP_output_{i}.nc", "r", format="NETCDF4")
    volt_array[:, (i-1)] = np.array(dat['volt'][:][:,0])
    dat.close()

#Calculate the 2.5th and 97.5th percentile of the voltage at each time step to get a 95% confidence
ana = np.percentile(volt_array, [2.5, 97.5], axis=1)

#Plot the range onto a graph
plt.fill_between(x, ana[0, :], ana[1, :], alpha=0.2, label='95% Confidence (random samps)', color='C0')
plt.plot(x, ana[0, :], color='C0', alpha=0.5)
plt.plot(x, ana[1, :], color='C0', alpha=0.5)

#Save the data to a data frame, and a .csv file for the user to be able to move to a different plotting software
dat_fram = {'mean volt': volt_dat_mu,
            '5th percentile': ana[0,:],
            '95th percentile': ana[1,:]}
df = pd.DataFrame(dat_fram, columns=['mean', '5th percentile', '95th percentile'])
df.to_csv('voltage_confidence_up.csv', index=False)

#Plot the mean voltage
plt.plot(x, volt_dat_mu, color='black', label='Mean Voltage')
plt.title('Uncertainty in Voltage Calculation', pad=15.0)
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')
plt.grid()

#If the user has performed the sensitivity analysis, and calculated uncertainty using dV_dx (see generate_inp_params.py)...
#...They can plot this data on top of the other uncertainty propagation data 
std_V_dat = pd.read_csv('uq_code/data_store_up/std_V_dat.csv')

std_V = np.array(std_V_dat['std_V'][:])
low_b = volt_dat_mu-(2*std_V)
up_b = volt_dat_mu+(2*std_V)
plt.fill_between(x, low_b, up_b, alpha=0.2, label='95% Confidence (SDs)', color='C3')
plt.plot(x, low_b, alpha=0.5, color='C3')
plt.plot(x, up_b, alpha=0.5, color='C3')

#Display the plot
plt.legend()

inp_dat.close()
dat_mu.close()

plt.show()

